In [1]:
from utils_project import *
from scipy.stats import shapiro, mannwhitneyu, bartlett, ttest_ind, chi2_contingency


Detected user: /Users/matty_gee
Base directory: /Volumes/synapse/projects/SocialSpace/Projects/SNT-fmri_place/Trajectory_analyses
Included n=50
Found 56 mask nifties


In [7]:
get_cols('sex')

,sex,mentalizing_initial_screening_sex_surgery,mentalizing_initial_screening_sex_surgery_when,mentalizing_initial_screening_sex_meds,mentalizing_initial_screening_sex_meds_list,staxi_sex,bdi_21_interestsex,ctq_sexual_abuse_score,bdi_21_interestsex_mri,leq_sexually_assaulted,leq_sexually_assaulted_detail,sex_int
0,F,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
1,F,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
2,M,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
3,M,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
4,M,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
5,M,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
6,F,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
7,F,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
8,F,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
9,F,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0


In [9]:
data['sex_int'][data['sex'].isin(['F','female'])] = 0
data['sex_int'][data['sex'].isin(['M','male'])] = 1

print(data[data["sample"] == 'Initial']['sex'].value_counts())
print(data[data["sample"] == 'Validation']['sex'].value_counts())

sex = [[17,15],[7,11]]
stat, p, dof, expected = chi2_contingency(sex)
print(f'Sex chi-squared stat={np.round(stat,3)} p={np.round(p,4)}')

M    10
F     8
Name: sex, dtype: int64
F    17
M    15
Name: sex, dtype: int64
Sex chi-squared stat=0.452 p=0.5014


In [11]:
init_df = data[data["sample"] == 'Initial']
val_df  = data[data["sample"] == 'Validation']
for measure in ['age_years']:
    
    print(measure) 
    print(f'original mean={np.round(np.mean(init_df[measure]),2)}, sd={np.round(np.std(init_df[measure]),2)}')
    print(f'repl mean={np.round(np.mean(val_df[measure]),2)}, sd={np.round(np.std(val_df[measure]),2)}')

    # test for normality
    b,p = shapiro(val_df[measure])
    if p < 0.05: # not normal
        m,p = mannwhitneyu(val_df[measure], init_df[measure])
        print(f'mannwhitneyu m={np.round(m, 2)}, p={np.round(p,7)}')
    else: # normal
        # test for equal variance
        b,p = bartlett(val_df[measure], init_df[measure])
        if p < 0.05: # not equal variances
            stat,p = ttest_ind(val_df[measure], init_df[measure], equal_var=False)
            print(f'Welchs t={np.round(stat, 5)} p={np.round(p,10)}')
        else: # equal variances
            stat,p = ttest_ind(val_df[measure], init_df[measure], equal_var=True)
            print(f't={np.round(stat, 5)} p={np.round(p,5)}')
    print('\n')

age_years
original mean=29.33, sd=3.42
repl mean=43.73, sd=10.05
Welchs t=7.24937 p=6.4e-09




In [ ]:
# print('lsas fear and age maybe are correlated')
# scipy.stats.pearsonr(x=all_df['lsas_fear_score'], y=all_df['age_years'])
# ols_df = pd.DataFrame(columns=['y', 'x', 'r2_adj', 'beta', 'p'])

# r = 0
# for m, measure in enumerate(['age_years', 'lsas_fear_score', 'sex_int']):
#     X = sm.add_constant(all_df[[measure] + ['sample']]).astype(float) #
#     for behav in ['power', 'affil']:
#         model = sm.OLS(all_df[behav + '_tendency_mean'], X, missing='drop')
#         reg = model.fit()
#         i = np.where(reg.pvalues.index == measure)[0][0]
#         ols_df.loc[r,:] = [behav, measure, reg.rsquared_adj, reg.params[i], reg.pvalues[i]]
#         r += 1
# ols_df

# for behav in ['power_tendency_mean', 'affil_tendency_mean', 'positive_tendency_mean', 'allo2d_area', 'ego2d_dists_mean_mean']:

#     X = sm.add_constant(all_df[['age_years','sex_int','sample']]).astype(float) #'age_years','sex_int','lsas_fear_score',
#     model = sm.OLS(all_df[behav], X, missing='drop')
#     reg = model.fit()
#     display(reg.summary())